## This HW depicts various models for training the provided Dataset - Rubab Karim

### Importing the Winequaity-red Dataset in order to conduct an analysis of the data

In [1]:
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import torch.optim as optim
import torch.nn as nn

In [2]:
wine_path = 'data/winequality-red.csv'
wineq_np = np.loadtxt(wine_path, dtype = np.float32, delimiter = ';', skiprows = 1)

t_c = torch.from_numpy(  wineq_np  )

# Reading the 'data/winequality-white.csv' file
white = pd.read_csv("data/winequality-white.csv", sep=';')

X = t_c[:, :-1]
Y = t_c[:, -1]

print(X.shape)
print(Y.shape)

torch.Size([1599, 11])
torch.Size([1599])


In [3]:
#t_c = torch.tensor(  t_c   )
#t_u = torch.tensor(  t_u   )

#print(t_u.shape)
#print(t_c.shape)

t_c = X.unsqueeze(1)
t_u = Y.unsqueeze(1)

print(t_u.shape)
print(t_c.shape)

torch.Size([1599, 1])
torch.Size([1599, 1, 11])


## After pre-processing, we split the dataset

In [4]:
n_samples = t_u.shape[0]
n_samples

1599

In [5]:
n_val = int(0.3 * n_samples)
n_val

479

In [6]:
shuffled_indeces = torch.randperm(n_samples)
shuffled_indeces

tensor([1426, 1337,  835,  ...,  662, 1482, 1190])

In [7]:
train_indeces = shuffled_indeces[:-n_val]
val_indeces = shuffled_indeces[-n_val:]

print(train_indeces)
print(val_indeces)

tensor([1426, 1337,  835,  ...,  954, 1579,  633])
tensor([ 622,  435,  140, 1047,   54,  209,  877, 1336,  654,  249, 1351,  287,
         198,  639, 1455,  953,  220, 1201, 1222,  250,  706,  432, 1037, 1407,
         826,  987, 1462, 1478,  945,  807,  787,  171,  939,  828,  789,  570,
         231,  195, 1306,  381, 1178, 1437,  601,   31,  676,  749, 1292,  352,
         424, 1598,  124,  150,  439,  137,  331,  273, 1551,  373, 1473,  929,
         357,   62,  821,  522,  537, 1464,  793,    1,  515,  715,  818, 1305,
        1074, 1529, 1516,  516, 1435,    8, 1113,  368,   72, 1349,  782,  261,
         131,  745,   92,  251,  702, 1122, 1191,   20,  858, 1032,  318,  634,
         510,  247,   41,   33, 1031,  224,  491,  390, 1133, 1246, 1229,  724,
         660,  478,  699,  178,  182, 1406,  983,  378,  327, 1027, 1483,  320,
         372,  890,  964, 1249, 1302, 1180, 1069, 1514,  384, 1228,  813,  513,
         736,   55,  705, 1509,  901,  791,  722,  341,  860, 1311,  

In [8]:
train_t_u = t_u[train_indeces]
train_t_c = t_c[train_indeces]

val_t_u = t_u[val_indeces]
val_t_c = t_c[val_indeces]

train_t_u = 0.1 * train_t_u
val_t_u   = 0.1 * val_t_u

## After prepping the dataset, we now push it into the Training Loop

In [9]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_t_u, val_t_u, train_t_c, val_t_c):
    
    for epoch in range(1, n_epochs):
        
        train_t_p = model(train_t_u)
        train_loss = loss_fn(train_t_p, train_t_c)
        
        val_t_p = model(val_t_u)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        
        
        if epoch % 50 == 0:
            print("Train Loss: ", train_loss.item())
            print("Val Loss: ", val_loss.item())

### Here, we define the Regression Models that are going to be used in this analysis. 

#### a) Linear Regression Model

In [10]:
def model_fn_linear():
    return nn.Linear(1, 1)

#### b) Multi-Layer Perceptron

In [11]:
def model_fn_nn_1hidden():
    
    seq_model = nn.Sequential(
        nn.Linear(1, 11),
        nn.ReLU(),
        
        nn.Linear(11, 1),
    
    )
    
   # optimizer = optim.SGD(
   #    model_fn_nn_1hidden.parameters(),
   #    lr=1e-3  )
    
    return seq_model

#### c) Deep Neural Network with 2 hidden layers

In [12]:
def model_deep_learning_2hidden():
    seq_model = nn.Sequential(
        nn.Linear(1, 15),
        nn.ReLU(),
        
        nn.Linear(15, 5),
        nn.Tanh(),
        
        nn.Linear(5, 1)
    )
    
    return seq_model

#### d) Deep Neural Network with 4 hidden layers

In [13]:
def model_deep_learning_4hidden():
    seq_model = nn.Sequential(
        nn.Linear(1, 20),
        nn.ReLU(),
        
        nn.Linear(20, 15),
        nn.Tanh(),
        
        nn.Linear(15, 10),
        nn.Tanh(),
        
        nn.Linear(10, 5),
        nn.Tanh(),
        
        nn.Linear(5, 1)
    )
    
    return seq_model

### Now, we call the function loop and push data into the training loop

In [14]:
#model = model_fn_linear()

model = model_fn_nn_1hidden()

#model = model_deep_learning_2hidden()

#model = model_deep_learning_4hidden()

optimizer = optim.Adam(model.parameters(), lr = 0.001)

loss_fn = nn.MSELoss().float()

In [15]:
result = training_loop(
    
    n_epochs = 1000,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_t_u = train_t_u,
    val_t_u = val_t_u,
    train_t_c = train_t_c,
    val_t_c = val_t_c
)

/opt/miniconda3/envs/ITS5200/lib/python3.8/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1120, 1, 11])) that is different to the input size (torch.Size([1120, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/miniconda3/envs/ITS5200/lib/python3.8/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([479, 1, 11])) that is different to the input size (torch.Size([479, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Train Loss:  334.5809631347656
Val Loss:  359.63262939453125
Train Loss:  329.7444152832031
Val Loss:  354.7177734375
Train Loss:  324.2481384277344
Val Loss:  349.1172790527344
Train Loss:  318.1285705566406
Val Loss:  342.8774108886719
Train Loss:  311.40130615234375
Val Loss:  336.0080261230469
Train Loss:  304.3565979003906
Val Loss:  328.80023193359375
Train Loss:  297.43206787109375
Val Loss:  321.6960754394531
Train Loss:  291.06982421875
Val Loss:  315.144287109375
Train Loss:  285.6182556152344
Val Loss:  309.5010070800781
Train Loss:  281.27691650390625
Val Loss:  304.9740295410156
Train Loss:  278.0794677734375
Val Loss:  301.60467529296875
Train Loss:  275.9128723144531
Val Loss:  299.2863464355469
Train Loss:  274.56817626953125
Val Loss:  297.8143615722656
Train Loss:  273.8055725097656
Val Loss:  296.9502868652344
Train Loss:  273.4100036621094
Val Loss:  296.4781188964844
Train Loss:  273.2217102050781
Val Loss:  296.23468017578125
Train Loss:  273.13885498046875
Val Lo

## 